# Face Recognition: Binary Image Classification Convolutional Neural Network

### Process training set to extract faces.. 

In [ ]:
# do image extraction i.e., just extract faces to increase accuracy.. this will exclude clothing 

# extract and save each detected face in a photograph
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
 
# save each face separately
def extract_faces(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot each face as a subplot
    for i in range(len(result_list)):
        confidence = result_list[i]['confidence']        
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        if confidence >= 0.95:
                pyplot.imsave('output/'+ filename, data[y1:y2, x1:x2])        

import glob
images = glob.glob("dataset/training_set/*/*.j*")
for image in images:
    filename = image
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    faces = detector.detect_faces(pixels)
    # display faces on the original image
    extract_faces(filename, faces)
    

2021-10-24 06:56:59.342613: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-24 06:56:59.705579: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


### Process test set to extract faces..

In [2]:
# do image extraction i.e., just extract faces to increase accuracy.. this will exclude clothing 

# extract and save each detected face in a photograph
def extract_faces(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot each face as a subplot
	for i in range(len(result_list)):
		# data = pyplot.imread(filename)
		confidence = result_list[i]['confidence']        
		# get coordinates
		x1, y1, width, height = result_list[i]['box']
		x2, y2 = x1 + width, y1 + height
		if confidence >= 0.95:
				pyplot.imsave('output/'+ filename, data[y1:y2, x1:x2])        


print(glob.glob("dataset/test_set/*/*.jp*"))
images = glob.glob("dataset/test_set/*/*.jp*")
for image in images:
    filename = image
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    faces = detector.detect_faces(pixels)
    # display faces on the original image
    extract_faces(filename, faces)

['dataset/test_set/Niyati/IMG_6664.jpg', 'dataset/test_set/Niyati/20150502_104836.jpeg', 'dataset/test_set/Niyati/20150516_223333.jpg', 'dataset/test_set/Niyati/DSC02597.jpeg', 'dataset/test_set/Niyati/IMG_8466.jpg', 'dataset/test_set/Niyati/20150516_223323.jpg', 'dataset/test_set/Niyati/IMG_0624.jpeg', 'dataset/test_set/Niyati/IMG_8464.jpg', 'dataset/test_set/Niyati/IMG_20150426_201701.jpg', 'dataset/test_set/Niyati/20171019_194210.jpeg', 'dataset/test_set/Niyati/IMG_7916.jpg', 'dataset/test_set/Niyati/IMG_1381.jpg', 'dataset/test_set/Niyati/20150507_202558.jpg', 'dataset/test_set/Niyati/IMG_8549.jpg', 'dataset/test_set/Niyati/IMG_0623.jpeg', 'dataset/test_set/Niyati/IMG_8429.jpg', 'dataset/test_set/Niyati/IMG_7915.jpg', 'dataset/test_set/Niyati/IMG_2129.jpg', 'dataset/test_set/Niyati/20150514_221720.jpg', 'dataset/test_set/Niyati/IMG_7456.jpg', 'dataset/test_set/Niyati/IMG_7536.jpg', 'dataset/test_set/Niyati/20150503_185608.jpg', 'dataset/test_set/Niyati/IMG_0115.jpeg', 'dataset/test

## Import Libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

### Load the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('output/dataset/training_set',
                                                 target_size = (256, 256),
                                                 batch_size = 10,
                                                 class_mode = 'binary')

### Load the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('output/dataset/test_set',
                                            target_size = (256, 256),
                                            batch_size = 10,
                                            class_mode = 'binary')

## Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[256, 256, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

In [ ]:
cnn.summary()

## Making predictions

In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import glob

def predict_person(filename):
    model = cnn
    import numpy as np
    from keras.preprocessing import image
    # N c
    test_image = image.load_img(filename, target_size = (256, 256))
    test_image = image.img_to_array(test_image)
    # model was training with a batch of images, so have to expand dimensions
    test_image = np.expand_dims(test_image, axis = 0)
    # feature scaling i.e., normalization
    result = model.predict(test_image/255.0)
    # find which index corresponds to which classification class
    # print(training_set.class_indices)
    # access the first batch and then the first image
    print(result[0][0])
    if result[0][0] > 0.5:
      prediction = 'Vanshika'
    else:
      prediction = 'Niyati'
    print(prediction)

# extract each face separately
def extract_faces(filename, result_list):
        # load the image
        data = pyplot.imread(filename)
        current_confidence = 0 
        for i in range(len(result_list)):
        # data = pyplot.imread(filename)
            confidence = result_list[i]['confidence']
            # this confidence comparison is done to ensure that if MTCNN identifies multiple faces then we take the one 
            # with the highest confidence.. our input will always have one person only in the image
            if confidence > current_confidence:
               # get coordinates
                           x1, y1, width, height = result_list[i]['box']
                           x2, y2 = x1 + width, y1 + height
                           pyplot.imsave('tmp/'+filename, data[y1:y2, x1:x2])   
                           pyplot.imshow(data[y1:y2, x1:x2])
                           pyplot.show() 
                           current_confidence = confidence 
        predict_person('tmp/'+filename)                         
                           
images = glob.glob("dataset/guess/*.jp*")
for image in images:
    print("********")
    filename = image
    print(image)    
    # load image from file
    pixels = pyplot.imread(filename)    
    pyplot.imshow(pixels)
    pyplot.show()
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    faces = detector.detect_faces(pixels)
    # display faces on the original image
    extract_faces(filename, faces)
    print("********")
    


### Save the model

In [25]:
cnn.save('imgClassification')

2021-10-18 13:45:25.008533: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: imgClassification/assets
